In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import csv
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [ ]:
# Mount Google Drive to access the data and save output
from google.colab import drive

drive.mount('/content/drive')
data_dir = "drive/MyDrive/SI_4lab/2D/data"
output_dir = "drive/MyDrive/SI_4lab/2D"
os.makedirs(output_dir, exist_ok=True)

np.random.seed(69)
torch.manual_seed(69)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(69)

# Determine the device to use (GPU if available, otherwise CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Mounted at /content/drive
Using device: cpu


In [ ]:
# Define a dictionary of hyperparameters
HYPERPARAMS = {
    # Data parameters
    'img_size': 224,       # Size to resize images to
    'batch_size': 20,      # Number of images per batch
    'test_size': 0.1,      # Proportion of data for the test set
    'val_size': 0.1,       # Proportion of data for the validation set

    # Training parameters
    'epochs': 50,          # Maximum number of training epochs
    'patience': 10,        # Number of epochs with no improvement before early stopping
    'learning_rate': 0.001, # Learning rate for the optimizer

    # Model parameters
    'architecture': 'simple',  # 'simple', 'medium', or 'complex' CNN architecture
    'dropout_rate': 0.5,   # Dropout rate for regularization
    'use_batch_norm': False, # Whether to use batch normalization
    'activation': 'relu',  # Activation function for hidden layers ('relu', 'tanh', 'elu', or 'selu')
    'optimizer_name': 'adam',  # Optimizer to use ('adam', 'sgd', or 'rmsprop')
    'kernel_size': 3,      # Kernel size for convolutional layers
    'num_filters': 32,     # Base number of filters in the first convolutional layer
    'pool_size': 2         # Size of the max pooling window
}

In [ ]:
# Function to define image transformations for training, validation, and testing
def get_transforms(img_size=HYPERPARAMS['img_size']):
    # Transformations for the training set (includes data augmentation)
    train_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),  # Resize images
        transforms.RandomHorizontalFlip(),       # Randomly flip images horizontally
        transforms.RandomRotation(10),          # Randomly rotate images by up to 10 degrees
        transforms.ToTensor(),                  # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with ImageNet stats
    ])

    # Transformations for validation and test sets (no data augmentation)
    val_test_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),  # Resize images
        transforms.ToTensor(),                  # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with ImageNet stats
    ])

    return train_transform, val_test_transform

In [ ]:
# Custom PyTorch Dataset for loading images from a list of file paths
class SplitImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths  # List of image file paths
        self.labels = labels            # List of corresponding labels
        self.transform = transform      # Image transformations to apply

    def __len__(self):
        # Return the total number of images in the dataset
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Get the image path and label for a given index
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB") # Open and convert image to RGB

        label = self.labels[idx]

        # Apply transformations if specified
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Function to prepare the data: load images, split into train/val/test sets, and create DataLoaders
def prepare_data(data_dir, batch_size=HYPERPARAMS['batch_size'],
                test_size=HYPERPARAMS['test_size'], val_size=HYPERPARAMS['val_size']):
    # Define directories for each class
    muffin_dir = os.path.join(data_dir, "muffin")
    chihuahua_dir = os.path.join(data_dir, "chihuahua")

    # Get image paths for each class (limit to 1000 for demonstration)
    muffin_paths = sorted([os.path.join(muffin_dir, f) for f in os.listdir(muffin_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])[:1000]
    chihuahua_paths = sorted([os.path.join(chihuahua_dir, f) for f in os.listdir(chihuahua_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])[:1000]

    # Create labels (0 for muffin, 1 for chihuahua)
    muffin_labels = [0] * len(muffin_paths)
    chihuahua_labels = [1] * len(chihuahua_paths)

    # Combine paths and labels
    all_paths = muffin_paths + chihuahua_paths
    all_labels = muffin_labels + chihuahua_labels

    # Split into train, validation, and test sets using stratified splitting
    X_temp, X_test, y_temp, y_test = train_test_split(
        all_paths, all_labels, test_size=test_size, stratify=all_labels, random_state=42
    )

    # Adjust validation size based on the remaining data after test split
    val_size_adjusted = val_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_size_adjusted, stratify=y_temp, random_state=42
    )

    # Get image transformations
    train_transform, val_test_transform = get_transforms()

    # Create custom datasets for each set
    train_dataset = SplitImageDataset(X_train, y_train, transform=train_transform)
    val_dataset = SplitImageDataset(X_val, y_val, transform=val_test_transform)
    test_dataset = SplitImageDataset(X_test, y_test, transform=val_test_transform)

    # Create DataLoaders for efficient batch processing
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader, X_test, y_test

In [ ]:
# Define the Convolutional Neural Network (CNN) model
class CNN(nn.Module):
    def __init__(self, architecture='simple', dropout_rate=0.5, activation='relu',
                 use_batch_norm=False, kernel_size=3, num_filters=32, pool_size=2):
        super(CNN, self).__init__()

        # Define activation function based on the chosen option
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'elu':
            self.activation = nn.ELU()
        elif activation == 'selu':
            self.activation = nn.SELU()
        else:
            self.activation = nn.ReLU()  # Default to ReLU

        # Define architecture configurations for simple, medium, and complex models
        configs = {
            'simple': {
                'conv_layers': 1,
                'dense_layers': [50]
            },
            'medium': {
                'conv_layers': 2,
                'dense_layers': [100]
            },
            'complex': {
                'conv_layers': 3,
                'dense_layers': [200, 100]
            }
        }

        # Get the configuration for the chosen architecture
        config = configs.get(architecture, configs['simple'])

        # Build the convolutional block
        layers = []
        in_channels = 3  # Input channels for RGB images

        for i in range(config['conv_layers']):
            out_channels = num_filters * (2**i) # Increase filters in deeper layers

            # Add Convolutional layer
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1))

            # Add Batch Normalization if enabled
            if use_batch_norm:
                layers.append(nn.BatchNorm2d(out_channels))

            # Add Activation function
            layers.append(self.activation)

            # Add Max Pooling layer
            layers.append(nn.MaxPool2d(pool_size))

            # Add Dropout after the last convolutional layer if enabled
            if i == config['conv_layers'] - 1 and dropout_rate > 0:
                layers.append(nn.Dropout2d(dropout_rate))

            in_channels = out_channels

        self.conv_block = nn.Sequential(*layers)

        # Calculate the size of the flattened output after convolutions
        # This assumes an initial image size of 224x224 and that each pooling reduces size by pool_size
        feature_size = 224 // (pool_size ** config['conv_layers'])
        flattened_size = in_channels * feature_size * feature_size

        # Build the dense (fully connected) layers
        dense_layers = []
        in_features = flattened_size

        for units in config['dense_layers']:
            dense_layers.append(nn.Linear(in_features, units))
            dense_layers.append(self.activation)
            # Add Dropout to dense layers if enabled
            if dropout_rate > 0:
                dense_layers.append(nn.Dropout(dropout_rate))
            in_features = units

        # Add the output layer (2 units for two classes: muffin and chihuahua)
        dense_layers.append(nn.Linear(in_features, 2))

        self.classifier = nn.Sequential(*dense_layers)

    # Define the forward pass of the model
    def forward(self, x):
        x = self.conv_block(x)       # Pass input through convolutional block
        x = torch.flatten(x, 1)    # Flatten the output for dense layers
        x = self.classifier(x)       # Pass flattened output through dense layers
        return x

In [ ]:
# Function to train the model
def train_model(model, train_loader, val_loader, optimizer, criterion,
                epochs=HYPERPARAMS['epochs'], patience=HYPERPARAMS['patience']):
    # Move the model to the appropriate device (GPU or CPU)
    model.to(device)

    best_val_accuracy = 0.0  # Keep track of the best validation accuracy
    epochs_no_improve = 0    # Counter for early stopping

    train_losses = []       # List to store training losses per epoch
    train_accuracies = []   # List to store training accuracies per epoch
    val_losses = []         # List to store validation losses per epoch
    val_accuracies = []     # List to store validation accuracies per epoch

    for epoch in range(epochs):
        # Training phase
        model.train()        # Set model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            # Move data to the device
            images, labels = images.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Update training loss
            running_loss += loss.item() * images.size(0)

            # Calculate training accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate average training loss and accuracy for the epoch
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_accuracy = correct / total

        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_accuracy)

        # Validation phase
        model.eval()         # Set model to evaluation mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient calculation during validation
        with torch.no_grad():
            for images, labels in val_loader:
                # Move data to the device
                images, labels = images.to(device), labels.to(device)

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Update validation loss
                running_loss += loss.item() * images.size(0)

                # Calculate validation accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate average validation loss and accuracy for the epoch
        epoch_val_loss = running_loss / len(val_loader.dataset)
        epoch_val_accuracy = correct / total

        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_accuracy)

        # Print epoch results
        print(f'Epoch {epoch+1}/{epochs} | '
              f'Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_accuracy:.4f} | '
              f'Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_accuracy:.4f}')

        # Early stopping logic
        if epoch_val_accuracy > best_val_accuracy:
            best_val_accuracy = epoch_val_accuracy
            epochs_no_improve = 0
            # Save the model state dictionary if it has the best validation accuracy
            torch.save(model.state_dict(), f'{output_dir}/best_model.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f'Early stopping after {epoch+1} epochs')
                break

    # Load the state dictionary of the best model after training
    model.load_state_dict(torch.load(f'{output_dir}/best_model.pth'))

    # Store training history
    history = {
        'train_loss': train_losses,
        'train_accuracy': train_accuracies,
        'val_loss': val_losses,
        'val_accuracy': val_accuracies
    }

    return model, history

In [ ]:
# Function to evaluate the model on the test set
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    model.to(device) # Move model to the device

    all_predicted = [] # List to store all predicted labels
    all_labels = []    # List to store all actual labels

    running_loss = 0.0
    correct = 0
    total = 0

    criterion = nn.CrossEntropyLoss() # Use Cross-Entropy Loss for evaluation

    # Disable gradient calculation during evaluation
    with torch.no_grad():
        for images, labels in test_loader:
            # Move data to the device
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Update test loss
            running_loss += loss.item() * images.size(0)

            # Calculate test accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Store predicted and actual labels
            all_predicted.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate average test loss and accuracy
    test_loss = running_loss / len(test_loader.dataset)
    test_accuracy = correct / total

    return test_loss, test_accuracy, all_predicted, all_labels

In [ ]:
# Function to plot training and validation results (accuracy and loss)
def plot_results(history, title=""):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot accuracy
    ax1.plot(history['train_accuracy'], label='Training accuracy')
    ax1.plot(history['val_accuracy'], label='Validation accuracy')
    ax1.set_title(f'Model accuracy {title}')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)

    # Plot loss
    ax2.plot(history['train_loss'], label='Training loss')
    ax2.plot(history['val_loss'], label='Validation loss')
    ax2.set_title(f'Model loss {title}')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
# Function to plot the confusion matrix and print classification report
def plot_confusion_matrix(y_true, y_pred, title=""):
    # Calculate the confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Plot the confusion matrix using seaborn heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues') # annot=True shows values, fmt='d' formats as integers
    plt.title(f'Confusion matrix {title}')
    plt.xlabel('Predicted class')
    plt.ylabel('Actual class')
    plt.tight_layout()
    plt.show()

    # Print the classification report
    print(classification_report(y_true, y_pred, target_names=['Muffin', 'Chihuahua']))

In [ ]:
# Function to save a sample of test predictions to a CSV file
def save_sample_predictions(X_test, y_test, y_pred, save_path):
    # Select 30 samples randomly, ensuring representation from both classes
    true_labels = np.array(y_test)
    predicted_labels = np.array(y_pred)

    # Find indices for each class in the test set
    muffin_indices = np.where(true_labels == 0)[0]
    chihuahua_indices = np.where(true_labels == 1)[0]

    # Select a specified number of samples from each class
    num_muffin = min(15, len(muffin_indices))
    num_chihuahua = min(15, len(chihuahua_indices))

    # Randomly select indices from each class
    selected_muffin = np.random.choice(muffin_indices, num_muffin, replace=False)
    selected_chihuahua = np.random.choice(chihuahua_indices, num_chihuahua, replace=False)

    # Combine the selected indices
    selected_indices = np.concatenate([selected_muffin, selected_chihuahua])

    # Prepare data for CSV
    csv_headers = ["Index", "Image Path", "Actual Label", "Predicted Label"]

    csv_rows = [csv_headers]
    for i, idx in enumerate(selected_indices):
        # Convert numerical labels back to class names
        actual = "Muffin" if true_labels[idx] == 0 else "Chihuahua"
        predicted = "Muffin" if predicted_labels[idx] == 0 else "Chihuahua"
        row = [i, X_test[idx], actual, predicted]
        csv_rows.append(row)

    # Save to CSV file
    with open(save_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(csv_rows)

    print(f"Sample predictions saved to: {save_path}")

In [ ]:
# Function to experiment with different hyperparameter values
def experiment_hyperparameter(
    train_loader, val_loader, param_name, param_values,
    architecture=HYPERPARAMS['architecture'],
    dropout_rate=HYPERPARAMS['dropout_rate'],
    use_batch_norm=HYPERPARAMS['use_batch_norm'],
    activation=HYPERPARAMS['activation'],
    optimizer_name=HYPERPARAMS['optimizer_name'],
    lr=HYPERPARAMS['learning_rate'],
    epochs=HYPERPARAMS['epochs'],
    batch_size=HYPERPARAMS['batch_size'],
    kernel_size=HYPERPARAMS['kernel_size'],
    num_filters=HYPERPARAMS['num_filters'],
    pool_size=HYPERPARAMS['pool_size']
):
    # Ensure the parameter name is supported for experimentation
    assert param_name in ['architecture', 'dropout_rate', 'use_batch_norm', 'activation', 'optimizer_name'], "Unsupported parameter"

    results = {} # Dictionary to store results for each parameter value

    for val in param_values:
        print(f"Training with {param_name}={val}...")

        # Create a new model instance with the current parameter value
        model_kwargs = {
            'architecture': architecture,
            'dropout_rate': dropout_rate,
            'use_batch_norm': use_batch_norm,
            'activation': activation,
            'kernel_size': kernel_size,
            'num_filters': num_filters,
            'pool_size': pool_size
        }

        # Update the specific parameter being experimented
        if param_name != 'optimizer_name': # Optimizer is not a model parameter
            model_kwargs[param_name] = val

        model = CNN(**model_kwargs)

        # Select the optimizer based on the parameter being experimented or the default
        curr_optimizer_name = optimizer_name
        if param_name == 'optimizer_name':
            curr_optimizer_name = val

        if curr_optimizer_name == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=lr)
        elif curr_optimizer_name == 'sgd':
            optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        elif curr_optimizer_name == 'rmsprop':
            optimizer = optim.RMSprop(model.parameters(), lr=lr)
        else:
            optimizer = optim.Adam(model.parameters(), lr=lr)  # Default to Adam

        criterion = nn.CrossEntropyLoss() # Loss function

        # Train the model with the current parameter value
        _, history = train_model(model, train_loader, val_loader, optimizer, criterion, epochs=epochs)

        # Store the training history and best validation metrics
        results[val] = {
            'history': history,
            'val_accuracy': max(history['val_accuracy']), # Best validation accuracy achieved
            'val_loss': min(history['val_loss']),       # Minimum validation loss achieved
        }

    # Plot the validation accuracy and loss for each parameter value
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    for val in param_values:
        plt.plot(results[val]['history']['val_accuracy'], label=f'{param_name}={val}')
    plt.title(f'Validation accuracy comparison ({param_name})')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    for val in param_values:
        plt.plot(results[val]['history']['val_loss'], label=f'{param_name}={val}')
    plt.title(f'Validation loss comparison ({param_name})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Find and print the parameter value that resulted in the best validation accuracy
    best_val = max(results, key=lambda x: results[x]['val_accuracy'])
    print(f"Best {param_name}: {best_val}")
    print(f"Validation accuracy: {results[best_val]['val_accuracy']:.4f}")
    print(f"Validation loss: {results[best_val]['val_loss']:.4f}")

    return results, best_val

In [ ]:
# Function to create and evaluate the final best model based on experiment results
def create_best_model(train_loader, val_loader, test_loader, X_test, y_test,
                      architecture, dropout_rate, use_batch_norm, activation, optimizer_name, lr=HYPERPARAMS['learning_rate']):
    # Create the best model with the optimal hyperparameters found
    best_model = CNN(
        architecture=architecture,
        dropout_rate=dropout_rate,
        use_batch_norm=use_batch_norm,
        activation=activation
    )

    # Select the best optimizer
    if optimizer_name == 'adam':
        optimizer = optim.Adam(best_model.parameters(), lr=lr)
    elif optimizer_name == 'sgd':
        optimizer = optim.SGD(best_model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_name == 'rmsprop':
        optimizer = optim.RMSprop(best_model.parameters(), lr=lr)
    else:
        optimizer = optim.Adam(best_model.parameters(), lr=lr) # Default

    criterion = nn.CrossEntropyLoss() # Loss function

    # Train the best model
    best_model, history = train_model(best_model, train_loader, val_loader, optimizer, criterion)

    # Plot the training and validation results for the best model
    plot_results(history, f"(Best model: {architecture}, {activation}, {optimizer_name})")

    # Evaluate the best model on the test set
    test_loss, test_accuracy, y_pred, y_true = evaluate_model(best_model, test_loader)
    print(f"Testing accuracy: {test_accuracy:.4f}")
    print(f"Testing loss: {test_loss:.4f}")

    # Plot the confusion matrix for the test set
    plot_confusion_matrix(y_true, y_pred, "Best model")

    # Save a sample of the test predictions
    save_sample_predictions(X_test, y_test, y_pred, f"{output_dir}/test_predictions.csv")

    return best_model, history

In [ ]:
if __name__ == "__main__":
    train_loader, val_loader, test_loader, X_test, y_test = prepare_data(
        data_dir=data_dir,
        batch_size=HYPERPARAMS['batch_size'],
        test_size=HYPERPARAMS['test_size'],
        val_size=HYPERPARAMS['val_size']
    )

    print(f"Number of training batches: {len(train_loader)}")
    print(f"Number of validation batches: {len(val_loader)}")
    print(f"Number of testing batches: {len(test_loader)}")

    # Experiment 1: Architecture comparison
    print("\nEXPERIMENT NR.1: ARCHITECTURE COMPARISON")
    arch_results, best_arch = experiment_hyperparameter(
        train_loader, val_loader,
        param_name='architecture',
        param_values=['simple', 'medium', 'complex']
    )

    # Experiment 2: Dropout layers comparison
    print("\nEXPERIMENT NR.2: DROPOUT LAYERS COMPARISON")
    dropout_results, best_dropout = experiment_hyperparameter(
        train_loader, val_loader,
        architecture=best_arch,
        param_name='dropout_rate',
        param_values=[0.0, 0.2, 0.5, 0.7]
    )

    # Experiment 3: Batch normalization comparison
    print("\nEXPERIMENT NR.3: BATCH NORMALISATION COMPARISON")
    bn_results, best_bn = experiment_hyperparameter(
        train_loader, val_loader,
        architecture=best_arch,
        dropout_rate=best_dropout,
        param_name='use_batch_norm',
        param_values=[False, True]
    )

    # Experiment 4: Activation functions comparison
    print("\nEXPERIMENT NR.4: ACTIVATION FUNCTIONS COMPARISON")
    act_results, best_activation = experiment_hyperparameter(
        train_loader, val_loader,
        architecture=best_arch,
        dropout_rate=best_dropout,
        use_batch_norm=best_bn,
        param_name='activation',
        param_values=['relu', 'tanh', 'elu', 'selu']
    )

    # Experiment 5: Optimizer comparison
    print("\nEXPERIMENT NR.5: OPTIMIZER COMPARISON")
    opt_results, best_optimizer = experiment_hyperparameter(
        train_loader, val_loader,
        architecture=best_arch,
        dropout_rate=best_dropout,
        use_batch_norm=best_bn,
        activation=best_activation,
        param_name='optimizer_name',
        param_values=['adam', 'sgd', 'rmsprop']
    )

    # Create and evaluate best model
    print("\nBEST MODEL TRAINING AND TESTING")
    best_model, best_history = create_best_model(
        train_loader, val_loader, test_loader, X_test, y_test,
        best_arch, best_dropout, best_bn, best_activation, best_optimizer
    )

    # Print best hyperparameters
    print("\nBEST HYPERPARAMETERS:")
    print(f"Architecture: {best_arch}")
    print(f"Dropout value: {best_dropout}")
    print(f"Batch normalization: {best_bn}")
    print(f"Activation function: {best_activation}")
    print(f"Optimizer: {best_optimizer}")

Number of training batches: 80
Number of validation batches: 10
Number of testing batches: 10

EXPERIMENT NR.4: ACTIVATION FUNCTIONS COMPARISON
Training with activation=relu...
Epoch 1/50 | Train Loss: 0.5922 | Train Acc: 0.7025 | Val Loss: 0.4961 | Val Acc: 0.7450
Epoch 2/50 | Train Loss: 0.4633 | Train Acc: 0.7944 | Val Loss: 0.3956 | Val Acc: 0.8650
Epoch 3/50 | Train Loss: 0.3824 | Train Acc: 0.8481 | Val Loss: 0.3550 | Val Acc: 0.8750
Epoch 4/50 | Train Loss: 0.3208 | Train Acc: 0.8706 | Val Loss: 0.3074 | Val Acc: 0.8800
Epoch 5/50 | Train Loss: 0.3023 | Train Acc: 0.8838 | Val Loss: 0.2740 | Val Acc: 0.9000
Epoch 6/50 | Train Loss: 0.2732 | Train Acc: 0.8950 | Val Loss: 0.2531 | Val Acc: 0.9000
Epoch 7/50 | Train Loss: 0.2307 | Train Acc: 0.9087 | Val Loss: 0.2391 | Val Acc: 0.9100
Epoch 8/50 | Train Loss: 0.2069 | Train Acc: 0.9175 | Val Loss: 0.2403 | Val Acc: 0.9250
Epoch 9/50 | Train Loss: 0.1966 | Train Acc: 0.9213 | Val Loss: 0.1887 | Val Acc: 0.9250
Epoch 10/50 | Train Lo

KeyboardInterrupt: 